In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import math
import csv
from csv import writer
import concurrent.futures
import random

In [ ]:
#scrape from freeproxy.world
url = "https://www.freeproxy.world/"
proxylist =[]
#filter through all pages
for i in range(1,75):
    print(i)
    try:
        #filter to http proxy
        querystring = {"type":"http","anonymity":"","country":"","speed":"","port":"","page":f"{i}"}

        payload = ""
        headers = {"User-Agent": "insomnia/8.4.5"}

        #site request
        site = requests.request("GET", url, data=payload, headers=headers, params=querystring)
        soup = BeautifulSoup(site.content, 'html.parser')
        trs = soup.find_all('tr')
        for tr in trs:
            try:
                ip = tr.find('td',class_=re.compile('show-ip-div')).text.strip()
                port = tr.find('a').text.strip()
                proxylist.append(f'{ip}:{port}')
            except:
                pass
    except:
        pass
print(len(proxylist))




#scrape free proxies from proxyscrape.com
headers = {"User-Agent": "insomnia/8.4.5"}
#site request
url = 'https://proxyscrape.com/free-proxy-list'
site = requests.get(url,headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape http proxy download link
divs = soup.find_all('div', class_=re.compile('itemcard downloadcard'))
for div in divs:
    h2 = div.find('h2')
    try:
        if(h2.text.strip()=='HTTP Proxies'):
            a = div.find('a')
            download = a['download_url']
    except:
        pass

#get content of download link
try:
    site = requests.get(download,headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    #add proxies to testlist
    tempproxylist = soup.text.strip().split('\r')
    for proxy in tempproxylist:
        proxylist.append(proxy)

    print(len(proxylist))
except:
    pass    
#scrape proxies from free-proxy-list.net
url = 'https://free-proxy-list.net/'
#site + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
#scrape table
try:
    table = soup.find('div', class_=re.compile('table-responsive'))
    trlabels = table.find_all('tr')
except:
    pass
#scrape proxies and add to test list
try:
    for i in trlabels:
            tdlabels = i.find_all('td')
            if(tdlabels[6].text.strip()=='yes'):
                proxylist.append(tdlabels[0].text.strip())
except:
    pass
print(len(proxylist))
#scrape proxies from hidemy.io
url = 'https://hidemy.io/en/proxy-list/'
#site request + soup
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
table = soup.find('div',class_=re.compile('table_block'))
#scrape table
trlabels = table.find_all('tr')
#scrape proxies and add to test list
for i in trlabels:
    tdlabels = i.find_all('td')
    try:

        if(tdlabels[4].text.strip()=='HTTP'):
            proxylist.append(tdlabels[0].text.strip())
    except:
        pass
print(len(proxylist))





#test proxies
workingProxies = []
def extract(proxy):
    works = False
    try:
        r = requests.get('https://www.whatismybrowser.com/detect/what-is-my-ip-address', proxies={'http':proxy, 'https':proxy}, timeout=2)
        soup = BeautifulSoup(r.content, 'html.parser')
        div = soup.find('div', class_='detected_result')
        ip = div.find('div').text.strip()
        print(f'{r}, {ip}')
        works = True
        workingProxies.append(proxy)
        with open('working_proxies.csv', 'w', encoding='UTF8', newline='') as workingProx:
            writer = csv.writer(workingProx)
            for k in workingProxies:
                writer.writerow([k])

    except:
        print(proxy)
    return works

#excecute test faster
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(extract, list(set(proxylist)))

In [2]:
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 16


In [9]:
url = 'https://www.landofbasketball.com/results/2021_2022_scores.htm'

payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)

<Response [200]>


In [11]:
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all('div', class_=re.compile('left margen-b8'))
print(links[0])

<div class="left margen-b8" style="width: 120px ; height: 60px">
<span><a href="2021_2022_oct_scores.htm">October</a></span>
<br>
					93 Games				</br></div>


In [13]:
a = links[0].find('a')
print(a)

<a href="2021_2022_oct_scores.htm">October</a>


In [14]:
part = a.get('href')
print(part)

2021_2022_oct_scores.htm


In [15]:
url = f'https://www.landofbasketball.com/results/{part}'
payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)

<Response [200]>


In [19]:
soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find_all('div', class_=re.compile('margin-t1 margen-b2 clearfix'))
print(links[0])

<div class="margin-t1 margen-b2 clearfix">
<div class="left">
<div class="left a-left a-right-sm" style="width: 180px; ">Milwaukee Bucks</div>
<div class="left a-right" style="width: 37px;"><a href="../box_scores/2021/1019BKNMIL.htm"> 127</a></div>
<div class="no-1 left margen-l2 margen-r2">-</div>
</div><div class="no-23-block" style="clear:both;"></div>
<div class="left">
<div class="left right-sm" style="width: 180px; ">Brooklyn Nets</div>
<div class="left a-right a-left-sm" style="width: 37px ; "><a href="../box_scores/2021/1019BKNMIL.htm"> 104</a></div></div>
</div>


In [21]:
a = links[0].find('a')
part = a.get('href')
print(part)

../box_scores/2021/1019BKNMIL.htm


In [22]:
url = f'https://www.landofbasketball.com/results/{part}'
payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})
print(response)

<Response [200]>


In [23]:
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-279138-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'UA-279138-1');
  </script>
  <script async="" nonce="W8zqgJDUWG9gCX0efOXIuQ" src="https://fundingchoicesmessages.google.com/i/pub-1808393148679576?ers=1">
  </script>
  <script nonce="W8zqgJDUWG9gCX0efOXIuQ">
   (function() {function signalGooglefcPresent() {if (!window.frames['googlefcPresent']) {if (document.body) {const iframe = document.createElement('iframe'); iframe.style = 'width: 0; height: 0; border: none; z-index: -1000; left: -1000px; top: -1000px;'; iframe.style.display = 'none'; iframe.name = 'googlefcPresent'; document.body.appendChild(iframe);} else {setTimeout(signalGooglefcPresent, 0);}}}signalGooglefcPresent();})();
  </script>
  <script async="" src="//pagead2.googlesyndication.com/pagead/j

In [60]:
date = None
awayTeam = None
homeTeam = None
awayTeam_record = None
homeTeam_record = None
awayTeam_points_total = None
homeTeam_points_total = None
awayTeam_points_q1 = None
homeTeam_points_q1 = None
awayTeam_points_q2 = None
homeTeam_points_q2 = None
awayTeam_points_q3 = None
homeTeam_points_q3 = None
awayTeam_points_q4 = None
homeTeam_points_q4 = None
awayTeam_PITP = None
homeTeam_PITP = None
awayTeam_FB_PTS = None
homeTeam_FB_PTS = None
awayTeam_BIG_LD = None
homeTeam_BIG_LD = None
awayTeam_BPTS = None
homeTeam_BPTS = None
awayTeam_TREB = None
homeTeam_TREB = None
awayTeam_TOV = None
homeTeam_TOV = None
awayTeam_TTOV = None
homeTeam_TTOV = None
awayTeam_POT = None
homeTeam_POT = None

In [61]:
dateHome = soup.find('div', class_=re.compile('pad-b10 pad-l5'))
dateHome = dateHome.text.strip().split('\n')
print(dateHome)

['Date: October 19, 2021', 'Home Team: Milwaukee Bucks']


In [62]:
#get Date
date = dateHome[0].split(': ')
date = date[1]
print(date)

October 19, 2021


In [63]:
#get home team
homeTeam = dateHome[1].split(': ')
homeTeam = homeTeam[1]
print(homeTeam)

Milwaukee Bucks


In [64]:
table = soup.find('table', class_=re.compile('size-11 a-left'))
tableText = table.text.strip().split('\n')
for i in range(len(tableText)):
    newI = tableText[i].replace('\t', '')
    tableText[i] = newI

filtered_text = [item for item in tableText if item]
filtered_text.pop(2)
print(filtered_text)


['104', 'Brooklyn Nets', '127', 'Milwaukee Bucks']


In [65]:
team1 = filtered_text[:2]
team2 = filtered_text[2:]

In [67]:
if homeTeam in team1:
    homeTeam_points_total = team1[0]
    awayTeam = team2[1]
    awayTeam_points_total = team2[0]
else:
    homeTeam_points_total = team2[0]
    awayTeam = team1[1]
    awayTeam_points_total = team1[0]

print(f'Home Team: {homeTeam}, PTS: {homeTeam_points_total}')
print(f'Away Team: {awayTeam}, PTS: {awayTeam_points_total}')



Home Team: Milwaukee Bucks, PTS: 127
Away Team: Brooklyn Nets, PTS: 104


In [72]:
quarterTable = soup.find('table', re.compile('by pad-y5 margen-b5'))
quarterStats = quarterTable.find_all('td')
qStatList = []
for stat in quarterStats:
    try:
        qStatList.append(int(stat.text.strip()))
    except:
        pass
print(qStatList)

[25, 34, 26, 19, 104, 37, 29, 31, 30, 127]


In [74]:
team1qStats = qStatList[5:]
team2qStats = qStatList[:5]
print(team1qStats)
print(team2qStats)

[37, 29, 31, 30, 127]
[25, 34, 26, 19, 104]


In [80]:
if homeTeam_points_total in str(team1qStats[4]):
    homeTeam_points_q1 = team1qStats[0]
    homeTeam_points_q2 = team1qStats[1]
    homeTeam_points_q3 = team1qStats[2]
    homeTeam_points_q4 = team1qStats[3]
    awayTeam_points_q1 = team2qStats[0]
    awayTeam_points_q2 = team2qStats[1]
    awayTeam_points_q3 = team2qStats[2]
    awayTeam_points_q4 = team2qStats[3]
if awayTeam_points_total in str(team1qStats[4]):
    homeTeam_points_q1 = team2qStats[0]
    homeTeam_points_q2 = team2qStats[1]
    homeTeam_points_q3 = team2qStats[2]
    homeTeam_points_q4 = team2qStats[3]
    awayTeam_points_q1 = team1qStats[0]
    awayTeam_points_q2 = team1qStats[1]
    awayTeam_points_q3 = team1qStats[2]
    awayTeam_points_q4 = team1qStats[3]
print(f'{homeTeam}: {homeTeam_points_q1}, {homeTeam_points_q2}, {homeTeam_points_q3}, {homeTeam_points_q4}')
print(f'{awayTeam}: {awayTeam_points_q1}, {awayTeam_points_q2}, {awayTeam_points_q3}, {awayTeam_points_q4}')

Milwaukee Bucks: 37, 29, 31, 30
Brooklyn Nets: 25, 34, 26, 19
